In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-07-23 00:02:56--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-23 00:02:56 (12.0 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json
import os

In [4]:
with open('documents.json') as f_in:
    doc_raw= json.load(f_in)

In [5]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course']= course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields =["question","text","section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [9]:
query='How do i enroll in the course?'

In [10]:
boost ={'question' : 3.0, 'section':0.5}
results = index.search(
    query= query,
    boost_dict =boost,
    num_results = 5,
    filter_dict= {'course':'data-engineering-zoomcamp'}
)


In [11]:
results

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make change

In [12]:
from openai import OpenAI

In [13]:
os.environ['OPENAI_API_KEY']

In [14]:
api_key = os.environ.get('OPENAI_API_KEY')

In [15]:
client = OpenAI(api_key = os.environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": query}
  ]
)
print(response.choices[0].message.content)

Enrolling in a course generally involves a few steps, which can vary depending on the institution or platform offering the course. Here’s a general guide to help you through the process:

### 1. Identify the Course:
- Visit the website of the educational institution or online learning platform.
- Search for the specific course you're interested in by using keywords related to the subject or by browsing through categories.

### 2. Check Prerequisites and Requirements:
- Read the course description to understand what is expected, including prerequisites, level (e.g., beginner, intermediate, advanced), and any required materials or tools.

### 3. Create an Account:
- Sign up for an account on the platform if you don’t already have one. You’ll typically need to provide basic information like your name, email address, and sometimes more detailed information depending on the platform.

### 4. Enroll in the Course:
- Look for an "Enroll," "Register," or "Sign Up" button on the course page.
- 

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:{context}
""".strip()

context = ""
    
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()

In [17]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do i enroll in the course?

CONTEXT:section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: How do I use Git / GitHub for this course?
answer: After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in thi

In [18]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [19]:
print(response.choices[0].message.content)

To enroll in the course, please follow these steps:

1. **Register before the course starts using this [link](#).**
2. **Subscribe to the course public Google Calendar** (this works only from Desktop).
3. **Join the course Telegram channel** for announcements.
4. **Register in DataTalks.Club's Slack** and join the channel.

This will ensure you are up-to-date with all course-related notifications and can participate in the community discussions.


In [20]:
def search(query):
    boost ={'question' : 3.0, 'section':0.5}
    search_result = index.search(
    query= query,
    boost_dict =boost,
    num_results = 5,
    filter_dict= {'course':'data-engineering-zoomcamp'}
)
    return search_result

In [21]:
def build_prompt(query, search_result):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:{context}
    """.strip()
    
    context = ""
        
    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
            
        prompt = prompt_template.format(question=query, context=context).strip()
        return prompt

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [23]:
query = 'how do i run kafka?'

def rag(query):
    search_result= search(query)
    prompt= build_prompt(query, search_result)
    answer = llm(prompt)
    return print(answer)

In [24]:
rag('the course has already started, can i still enroll?')

Yes, you can still enroll in the course after the start date. However, keep in mind that there will be deadlines for submitting the final projects, so make sure not to leave everything until the last minute.


In [28]:
from elasticsearch import Elasticsearch

In [29]:
es_client = Elasticsearch('http://localhost:9200')

In [30]:
es_client.info()

ObjectApiResponse({'name': '8e6da9bd0507', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tq8YZcGFSVe3DMDJj-7jfw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [31]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name ="course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [32]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [33]:
query= 'the course has already started, can i still enroll?'

In [34]:
def elastic_search(query):
    search_query= {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    results_doc=[]
    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])
    return results_doc

In [35]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [36]:
def rag(query):
    search_result= elastic_search(query)
    prompt= build_prompt(query, search_result)
    answer = llm(prompt)
    return print(answer)

In [37]:
rag(query)

Yes, you can still enroll in the course after it has started. You are eligible to submit the homework, but please be mindful of the deadlines for the final projects.
